# 🐸 Notebook de Evaluación del Sistema de Detección de Ranas

Este notebook permite:
1. Cargar y evaluar el modelo VAE entrenado
2. Visualizar el espacio latente
3. Analizar el rendimiento del detector
4. Generar visualizaciones para la tesis

In [ ]:
# Imports
import sys
sys.path.append('../models')
sys.path.append('../scripts')

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json

from vae_model import AudioVAE
from audio_processor import AudioProcessor
from detector import FrogDetector

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 1. Cargar Modelo y Configuración

In [ ]:
# Rutas
MODEL_PATH = '../trained_models/best_model.pth'
CONFIG_PATH = '../trained_models/detector_config.json'
FROG_DATA = '../data/processed'
OTHER_DATA = '../data/other_sounds'

# Cargar configuración
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("Configuración del Modelo:")
print(f"  Latent dim: {config['latent_dim']}")
print(f"  Radio: {config['radius']:.4f}")
print(f"  Sample rate: {config['model_config']['sample_rate']} Hz")
print(f"  Mel bins: {config['model_config']['n_mels']}")

In [ ]:
# Cargar modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

model = AudioVAE(
    input_shape=tuple(config['model_config']['input_shape']),
    latent_dim=config['latent_dim']
).to(device)

model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

print(f"✓ Modelo cargado: {sum(p.numel() for p in model.parameters()):,} parámetros")

## 2. Analizar Espacio Latente

In [ ]:
# Cargar vectores latentes guardados
all_latents = np.load('../trained_models/all_latents.npy')
centroid = np.array(config['centroid'])
radius = config['radius']

print(f"Vectores latentes cargados: {all_latents.shape}")
print(f"Centroide shape: {centroid.shape}")

# Calcular estadísticas
distances = np.linalg.norm(all_latents - centroid, axis=1)

print(f"\nEstadísticas de Distancias:")
print(f"  Media: {distances.mean():.4f}")
print(f"  Std: {distances.std():.4f}")
print(f"  Min: {distances.min():.4f}")
print(f"  Max: {distances.max():.4f}")
print(f"  Radio: {radius:.4f}")

In [ ]:
# Visualizar distribución de distancias
plt.figure(figsize=(12, 5))

# Histograma
plt.subplot(1, 2, 1)
plt.hist(distances, bins=50, alpha=0.7, color='green', edgecolor='darkgreen')
plt.axvline(radius, color='red', linestyle='--', linewidth=2, label=f'Radio = {radius:.4f}')
plt.xlabel('Distancia al Centroide')
plt.ylabel('Frecuencia')
plt.title('Distribución de Distancias')
plt.legend()
plt.grid(True, alpha=0.3)

# Box plot
plt.subplot(1, 2, 2)
plt.boxplot(distances, vert=True, patch_artist=True, 
            boxprops=dict(facecolor='green', alpha=0.7))
plt.axhline(radius, color='red', linestyle='--', linewidth=2, label=f'Radio = {radius:.4f}')
plt.ylabel('Distancia al Centroide')
plt.title('Box Plot de Distancias')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 3. Evaluar Detector

In [ ]:
# Crear detector
detector = FrogDetector(MODEL_PATH, CONFIG_PATH, device=str(device))
print("✓ Detector inicializado")

In [ ]:
# Evaluar en dataset de ranas
print("Evaluando en dataset de ranas...")
frog_results = detector.batch_detect(FROG_DATA)

frog_stats = detector.get_statistics(frog_results)

print(f"\nResultados en RANAS:")
print(f"  Total archivos: {frog_stats['total_files']}")
print(f"  Detectadas como rana: {frog_stats['frogs_detected']}")
print(f"  Tasa de detección: {frog_stats['detection_rate']*100:.1f}%")
print(f"  Distancia promedio: {frog_stats['avg_distance']:.4f}")

In [ ]:
# Evaluar en otros sonidos (si existen)
import os
if os.path.exists(OTHER_DATA) and len(list(Path(OTHER_DATA).glob('*.wav'))) > 0:
    print("\nEvaluando en otros sonidos...")
    other_results = detector.batch_detect(OTHER_DATA)
    
    other_stats = detector.get_statistics(other_results)
    
    print(f"\nResultados en OTROS SONIDOS:")
    print(f"  Total archivos: {other_stats['total_files']}")
    print(f"  Detectadas como rana (falsos positivos): {other_stats['frogs_detected']}")
    print(f"  Tasa de falsos positivos: {other_stats['detection_rate']*100:.1f}%")
    print(f"  Distancia promedio: {other_stats['avg_distance']:.4f}")
else:
    print("\n⚠ No hay otros sonidos para evaluar")
    other_results = []

## 4. Visualización de Espectrogramas

In [ ]:
# Crear procesador
processor = AudioProcessor(
    sample_rate=config['model_config']['sample_rate'],
    n_mels=config['model_config']['n_mels'],
    duration=config['model_config']['duration'],
    target_shape=(config['model_config']['n_mels'], config['model_config']['n_mels'])
)

# Visualizar algunos espectrogramas de ejemplo
frog_files = list(Path(FROG_DATA).glob('*.wav'))[:3]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, audio_file in enumerate(frog_files):
    spec = processor.process_audio_file(str(audio_file))
    
    axes[i].imshow(spec, aspect='auto', origin='lower', cmap='viridis')
    axes[i].set_title(f'Rana {i+1}')
    axes[i].set_xlabel('Time')
    axes[i].set_ylabel('Mel Frequency')

plt.tight_layout()
plt.suptitle('Ejemplos de Espectrogramas de Ranas', y=1.02, fontsize=14, fontweight='bold')
plt.show()

## 5. Reconstrucción del VAE

In [ ]:
# Seleccionar un audio de ejemplo
test_file = frog_files[0]
print(f"Probando reconstrucción con: {test_file.name}")

# Procesar
spec_tensor = processor.process_audio_to_tensor(str(test_file))
spec_tensor = spec_tensor.unsqueeze(0).to(device)

# Forward pass
with torch.no_grad():
    recon, mu, logvar = model(spec_tensor)

# Convertir a numpy
original = spec_tensor.cpu().numpy().squeeze()
reconstructed = recon.cpu().numpy().squeeze()
latent = mu.cpu().numpy().squeeze()

# Visualizar
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

im1 = axes[0].imshow(original, aspect='auto', origin='lower', cmap='viridis')
axes[0].set_title('Espectrograma Original')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Mel Frequency')
plt.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(reconstructed, aspect='auto', origin='lower', cmap='viridis')
axes[1].set_title('Reconstrucción del VAE')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Mel Frequency')
plt.colorbar(im2, ax=axes[1])

# Diferencia
diff = np.abs(original - reconstructed)
im3 = axes[2].imshow(diff, aspect='auto', origin='lower', cmap='hot')
axes[2].set_title('Error Absoluto')
axes[2].set_xlabel('Time')
axes[2].set_ylabel('Mel Frequency')
plt.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.show()

print(f"\nVector latente (primeras 8 dimensiones): {latent[:8]}")
print(f"MSE: {np.mean((original - reconstructed)**2):.6f}")

## 6. Análisis de Detección

In [ ]:
# Analizar algunas detecciones específicas
print("Ejemplos de RANAS DETECTADAS CORRECTAMENTE:")
correct_detections = [r for r in frog_results if r['is_frog']][:5]
for r in correct_detections:
    print(f"  ✓ {r['filename'][:30]:30} | Dist: {r['distance']:.4f} | Conf: {r['confidence']:.2f}")

print(f"\nEjemplos de RANAS NO DETECTADAS (falsos negativos):")
missed_detections = [r for r in frog_results if not r['is_frog']][:5]
if missed_detections:
    for r in missed_detections:
        print(f"  ✗ {r['filename'][:30]:30} | Dist: {r['distance']:.4f} | Conf: {r['confidence']:.2f}")
else:
    print("  ¡Ninguno! Todas las ranas fueron detectadas correctamente.")

## 7. Métricas de Clasificación

In [ ]:
# Si tenemos datos de ambas clases
if len(other_results) > 0:
    from sklearn.metrics import confusion_matrix, classification_report
    
    # Crear labels verdaderos y predicciones
    true_labels = [1] * len(frog_results) + [0] * len(other_results)
    predictions = [1 if r['is_frog'] else 0 for r in frog_results] + \
                  [1 if r['is_frog'] else 0 for r in other_results]
    
    # Matriz de confusión
    cm = confusion_matrix(true_labels, predictions)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['No Rana', 'Rana'],
                yticklabels=['No Rana', 'Rana'])
    plt.ylabel('Verdadero')
    plt.xlabel('Predicho')
    plt.title('Matriz de Confusión')
    plt.show()
    
    # Reporte de clasificación
    print("\nReporte de Clasificación:")
    print(classification_report(true_labels, predictions, 
                               target_names=['No Rana', 'Rana']))
else:
    print("⚠ No hay datos de 'otros sonidos' para calcular métricas completas")

## 8. Guardar Figuras para Tesis

In [ ]:
# Crear directorio para figuras
FIGURES_DIR = Path('../thesis_figures')
FIGURES_DIR.mkdir(exist_ok=True)

print(f"Guardando figuras en: {FIGURES_DIR}")
print("\nPara generar las visualizaciones del espacio latente, ejecuta:")
print("python ../scripts/visualize_latent_space.py \\")
print("  --model-path ../trained_models/best_model.pth \\")
print("  --config-path ../trained_models/detector_config.json \\")
print("  --frog-data ../data/processed \\")
print("  --other-data ../data/other_sounds \\")
print("  --output-dir ../thesis_figures")

## 9. Conclusiones

Este notebook permite:
- ✅ Evaluar el rendimiento del detector
- ✅ Visualizar reconstrucciones del VAE
- ✅ Analizar el espacio latente
- ✅ Calcular métricas de clasificación
- ✅ Generar figuras para la tesis

**Próximos pasos:**
1. Ejecutar visualize_latent_space.py para generar visualizaciones 2D/3D
2. Analizar falsos positivos/negativos
3. Ajustar hiperparámetros si es necesario
4. Documentar resultados en la tesis